In this notebook, we'll learn how to develop a simple flower image classification model, convert it to CoreML model format and then deploy this model into an iOS app for static as well as real time image classification.

So, let's get started.

## Step-1: Dataset loading, Processing and Analysis

In this notebook, we'll be using turicreate framework provided by apple as n open source project on Github and has a lot of tools built-in for doing at-least the basic stuff such as image classification, recommendation, style transfer, activity recognition etc.

For more insights, please see turicreate's Github repository: https://github.com/apple/turicreate

In [1]:
# Import Dependencies
import turicreate as tc

In [2]:
# Load Flower Images Dataset
data = tc.image_analysis.load_images('./dataset/flower_photos', with_path=True)

In [3]:
# Let's see what does this DataFrame "data" looks like
data

path,image
./dataset/flower_photos/daisy/100080576_f52e8e ...,Height: 263 Width: 320
./dataset/flower_photos/daisy/10140303196_b88d ...,Height: 313 Width: 500
./dataset/flower_photos/daisy/10172379554_b296 ...,Height: 215 Width: 320
./dataset/flower_photos/daisy/10172567486_2748 ...,Height: 333 Width: 500
./dataset/flower_photos/daisy/10172636503_21be ...,Height: 250 Width: 320
./dataset/flower_photos/daisy/102841525_bd6628 ...,Height: 400 Width: 500
./dataset/flower_photos/daisy/1031799732_e7f40 ...,Height: 375 Width: 500
./dataset/flower_photos/daisy/10391248763_1d16 ...,Height: 224 Width: 320
./dataset/flower_photos/daisy/10437754174_22ec ...,Height: 240 Width: 171
./dataset/flower_photos/daisy/10437770546_8bb6 ...,Height: 240 Width: 240


As we can see, data is a dataframe that contains two columns:

1. path: The path to all the images in the flower_photos folder
2. image: The dimensions of each image in the dataset.

In [4]:
# let's define an array with the names of all the flowers in the dataset
labels = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips'] 

In [5]:
# Function to get image labels
def get_image_labels(img_path=None, labels=labels):
    for label in labels:
        if label in img_path:
            return label

In [6]:
# Let's create a Label column using the path to images we got above
data['label'] = data['path'].apply(get_image_labels)

In [7]:
# Let's see what our DataFrame looks like after adding the labels column
data

path,image,label
./dataset/flower_photos/daisy/100080576_f52e8e ...,Height: 263 Width: 320,daisy
./dataset/flower_photos/daisy/10140303196_b88d ...,Height: 313 Width: 500,daisy
./dataset/flower_photos/daisy/10172379554_b296 ...,Height: 215 Width: 320,daisy
./dataset/flower_photos/daisy/10172567486_2748 ...,Height: 333 Width: 500,daisy
./dataset/flower_photos/daisy/10172636503_21be ...,Height: 250 Width: 320,daisy
./dataset/flower_photos/daisy/102841525_bd6628 ...,Height: 400 Width: 500,daisy
./dataset/flower_photos/daisy/1031799732_e7f40 ...,Height: 375 Width: 500,daisy
./dataset/flower_photos/daisy/10391248763_1d16 ...,Height: 224 Width: 320,daisy
./dataset/flower_photos/daisy/10437754174_22ec ...,Height: 240 Width: 171,daisy
./dataset/flower_photos/daisy/10437770546_8bb6 ...,Height: 240 Width: 240,daisy


Well it looks like that worked. Now we have a DataFrame that has the path to images, size of images as well as the label for each image. Next, let's save this dataframe and explore this data further.

In [8]:
# Saving Modified DataFrame "data"
data.save("flowers.sframe")

In [9]:
# Explore dataset
data.explore()

Materializing SFrame

## Step-2: Training the Image Classification Model

In [10]:
# Load the Saed DataFrame
df = tc.load_sframe('flowers.sframe')

In [11]:
df.head()

path,image,label
./dataset/flower_photos/daisy/100080576_f52e8e ...,Height: 263 Width: 320,daisy
./dataset/flower_photos/daisy/10140303196_b88d ...,Height: 313 Width: 500,daisy
./dataset/flower_photos/daisy/10172379554_b296 ...,Height: 215 Width: 320,daisy
./dataset/flower_photos/daisy/10172567486_2748 ...,Height: 333 Width: 500,daisy
./dataset/flower_photos/daisy/10172636503_21be ...,Height: 250 Width: 320,daisy
./dataset/flower_photos/daisy/102841525_bd6628 ...,Height: 400 Width: 500,daisy
./dataset/flower_photos/daisy/1031799732_e7f40 ...,Height: 375 Width: 500,daisy
./dataset/flower_photos/daisy/10391248763_1d16 ...,Height: 224 Width: 320,daisy
./dataset/flower_photos/daisy/10437754174_22ec ...,Height: 240 Width: 171,daisy
./dataset/flower_photos/daisy/10437770546_8bb6 ...,Height: 240 Width: 240,daisy


In [12]:
# Do a random Train-Test Split
train_data, test_data = df.random_split(0.8)

In [14]:
# Confirming the size of Training and Test Data
len(train_data), len(test_data)

(2931, 739)

In [15]:
# Create and Train the Model
clf_model = tc.image_classifier.create(dataset= train_data, target= 'label', max_iterations= 1000)

Download completed: /var/folders/4h/q_d_80nj6_bgdtsbx5_d851r0000gn/T/model_cache/resnet-50-symbol.json
Download completed: /var/folders/4h/q_d_80nj6_bgdtsbx5_d851r0000gn/T/model_cache/resnet-50-0000.params


Analyzing and extracting image features.

+------------------+--------------+------------------+

| Images Processed | Elapsed Time | Percent Complete |

+------------------+--------------+------------------+

...

| 225       | 305      | 1.000000  | 27.200771    | 0.999641          | 0.923611            |

| 250       | 336      | 1.000000  | 30.185079    | 0.999641          | 0.923611            |

| 258       | 350      | 1.000000  | 31.321656    | 0.999641          | 0.923611            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

So, what did we just do up there. We use the built-in function in turicrete to create an image classification model.  Hence, when it's provided with the training data, max number of iterations and what the prediction target is, it downloads the ResNet500 model and trains for our custom dataset using Transfer Learning.

As we can see above, this leads to a faster training and reaches a Training Accuracy of 99.96% within minutes.

In [16]:
# Let's test the model on our test data and see the predictions
model_predictions = clf_model.classify(test_data)

In [17]:
model_predictions

class,probability
daisy,0.999998487566036
daisy,0.9999998785463367
daisy,0.9809817066589522
daisy,0.9999999999133353
daisy,0.9999958804957375
daisy,0.9999999770550819
daisy,0.9999825113249032
daisy,0.9958535525327769
tulips,0.7669143758345719
daisy,0.9999999999539115


In [18]:
# Let's now evaluate our trained model and save the reults into a dictionary
results = clf_model.evaluate(test_data)
print ("Accuracy         : %s" % results['accuracy'])
print ("Confusion Matrix : \n%s" % results['confusion_matrix'])

Analyzing and extracting image features.

+------------------+--------------+------------------+

| Images Processed | Elapsed Time | Percent Complete |

+------------------+--------------+------------------+

| 64               | 1.96s        | 8.25%            |

...

| 320              | 12.85s       | 41.5%            |

| 640              | 26.58s       | 83.25%           |

| 739              | 32.02s       | 100%             |

+------------------+--------------+------------------+

Accuracy         : 0.8958051420838972
Confusion Matrix : 
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|  dandelion   |      tulips     |   6   |
|    daisy     |      roses      |   3   |
|    roses     |      roses      |  109  |
|  sunflowers  |      roses      |   3   |
|    tulips    |      daisy      |   3   |
|    roses     |      tulips     |   10  |
|    daisy     |    dandelion    |   7   |
|    daisy     |      tulips     |   5   |
|    daisy     |      daisy      |  119  |
|  dandelion   |    sunflowers   |   5   |
+--------------+-----------------+-------+
[21 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


So, we see that the trained model has an accuracy of about 89.58% on the test set. The confusion matrix shows the labels that were wrongly predicted and their count.

In [19]:
# let's save this model
clf_model.save('Flowers.model')

## Step-3: Convert Trained Model to CoreML Model

In [20]:
# Load the Saved model
model = tc.load_model('Flowers.model')

# Export the model to CoreML format
model.export_coreml('Flowers.mlmodel')  

This marks the end of Phase-1 for this tutorial. What we have done here is train a model, save the model and export it to CoreML format for deployment on iOS. The next step, Phase-2 is witing the iOS app and deploying this model for static as well as real-time flower classification.